In [1]:
#Gather relevant keys from our Secret Scope
ServicePrincipalID = dbutils.secrets.get(scope = "Analysts", key = "SPID")
ServicePrincipalKey = dbutils.secrets.get(scope = "Analysts", key = "SPKey")
DirectoryID = dbutils.secrets.get(scope = "Analysts", key = "DirectoryID")
DBUser = dbutils.secrets.get(scope = "Analysts", key = "DBUser")
DBPassword = dbutils.secrets.get(scope = "Analysts", key = "DBPword")


#Combine DirectoryID into full string
Directory = "https://login.microsoftonline.com/{}/oauth2/token".format(DirectoryID)

#Configure our ADLS Gen 2 connection with our service principal details
spark.conf.set("fs.azure.account.auth.type", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id", ServicePrincipalID)
spark.conf.set("fs.azure.account.oauth2.client.secret", ServicePrincipalKey)
spark.conf.set("fs.azure.account.oauth2.client.endpoint", Directory)

### Data Profiling Basics

One of the first jobs when developing a new data pipeline is to understand the data that you're working with. We'll briefly look at three different methods of getting to grips with a new dataset:
 - Using HEAD to peek at a file before trying to load it
 - Using the inbuilt describe() function
 - Importing specialist profiling libraries such as pandas_profiling

In [3]:
# Create DataFrame using one of our Taxi files
Sampledf = (spark
             .read
             .option("header","true")
             .option("inferSchema","true")
             .csv("/mnt/taxi/taxiFull/fhv_tripdata_2017-01.csv")
           )

display(Sampledf.describe())

summary,Dispatching_base_num,PUlocationID,DOlocationID
count,13657212,9952802,106013
mean,null,143.67607021620645,3.4901380019431577E-4
stddev,null,74.64678850663316,0.11363762848277774
min,B00001,0,0
max,b02881,265,37


In [4]:
import spark_df_profiling as spkprof

taxiProfiling = spkprof.ProfileReport(Sampledf)


In [5]:
str(taxiProfiling)

Out[ 46 ]: 'Output written to file /tmp/profile.html'

In [6]:
dbutils.fs.ls("/tmp/")

Out[ 47 ]: [FileInfo(path='dbfs:/tmp/hive/', name='hive/', size=0)]